In [1]:
import numpy as np

In [24]:
np.random.seed(64)
n_rows = 50
seq = np.random.random_integers(1, 64, n_rows) ** 1/np.random.random_integers(2, 9, n_rows)
data = np.array([seq,
        np.sin(seq),
        np.tanh(seq) + np.random.random_sample(size=n_rows),
        seq**(1/1.9)]).T
target = np.array([1 if (i % 3 == 0) else 0 for i in range(0, 50)])

/var/folders/51/6b4306h90693z7j7cd0g40kr0000gn/T/ipykernel_13866/3422886198.py:3: DeprecationWarning: This function is deprecated. Please call randint(1, 64 + 1) instead
  seq = np.random.random_integers(1, 64, n_rows) ** 1/np.random.random_integers(2, 9, n_rows)
/var/folders/51/6b4306h90693z7j7cd0g40kr0000gn/T/ipykernel_13866/3422886198.py:3: DeprecationWarning: This function is deprecated. Please call randint(2, 9 + 1) instead
  seq = np.random.random_integers(1, 64, n_rows) ** 1/np.random.random_integers(2, 9, n_rows)
